In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
sys.path.append("ext_deps/Hierarchical-Localization")
import pycolmap
from pathlib import Path
from imc2023.utils.utils import create_submission, get_data_from_dir, setup_logger
from imc2023.utils.eval import eval
from collections import defaultdict
from hloc.utils.io import list_h5_names
from hloc import localize_sfm, visualization
import os
import numpy as np
import logging

setup_logger()

plt_logger = logging.getLogger("matplotlib")
plt_logger.setLevel(logging.WARNING)

In [ ]:
dataset, scene = 'haiper', 'bike'
data_dir = Path('../image-matching-challenge-2023')
experiment_dir = Path(f'../outputs/alikedn16-rot-pixsfm-sci/{dataset}/{scene}')
sfm_path = experiment_dir / 'sparse'
features = experiment_dir / 'features.h5'
matches = experiment_dir / 'matches.h5'
pairs = experiment_dir / 'pairs.txt'

image_dir = data_dir / 'train' / dataset / scene / 'images'

reconstruction = pycolmap.Reconstruction(sfm_path)
print(reconstruction.summary())

data_dict = get_data_from_dir(data_dir, 'train')

img_list = list_h5_names(features)

# img_list = [x.replace(f'{dataset}/{scene}/', '') for x in data_dict[dataset][scene]]
reg_image_names = [im.name for imid, im in reconstruction.images.items()]

missing = list(set(img_list) - set(reg_image_names))
queries = [(x, pycolmap.infer_camera_from_image(image_dir / x)) for x in missing]

In [ ]:
results = defaultdict(lambda: defaultdict(dict))
for _, im in reconstruction.images.items():
    img_name = os.path.join(dataset, scene, "images", im.name)
    # problem: tvec is a reference! --> force copy
    results[dataset][scene][img_name] = {"R": im.rotmat(), "t": np.array(im.tvec)}
create_submission(results, data_dict, 'submission.csv')
scores_dict = eval(submission_csv='submission.csv', data_dir=data_dir, verbose=False, return_dict=True)

logging.info(scores_dict[dataset][scene])

In [ ]:
logs = localize_sfm.main(
    reconstruction,
    queries,
    pairs,
    features,
    matches,
    experiment_dir / 'loc.txt',
    covisibility_clustering=True)  # not required with SuperPoint+SuperGlue



In [ ]:
# results = defaultdict(lambda: defaultdict(dict))
for q, v in logs['loc'].items():
    v = v['log_clusters'][v['best_cluster']]
    print(q, v['PnP_ret']['num_inliers'], np.mean(list(v['PnP_ret']['inliers'])), len(v['db']))
    im = pycolmap.Image(q, tvec=v['PnP_ret']['tvec'], qvec=v['PnP_ret']['qvec'])
#     print(im.tvec, im.qvec)
    img_name = os.path.join(dataset, scene, "images", q)
    results[dataset][scene][img_name] = {"R": im.rotmat(), "t": np.array(im.tvec)}

create_submission(results, data_dict, 'submission.csv')

scores_dict = eval(submission_csv='submission.csv', data_dir=data_dir, verbose=False, return_dict=True)

logging.info(scores_dict[dataset][scene])

In [ ]:
visualization.visualize_loc(
    experiment_dir / 'loc.txt', image_dir, reconstruction, n=2, top_k_db=3, seed=10)